## Snowflake Connector 설정

In [1]:
%pip install --upgrade pip
%pip install --upgrade snowflake-connector-python

  Obtaining dependency information for pip from https://files.pythonhosted.org/packages/8a/6a/19e9fe04fca059ccf770861c7d5721ab4c2aebc539889e97c7977528a53b/pip-24.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 5.8 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 23.2.1
    Uninstalling pip-23.2.1:
      Successfully uninstalled pip-23.2.1
Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 938.4/938.4 kB 5.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 5.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 6.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


## Snowflake 연결 & 설치 테스트

In [1]:
import os

import snowflake.connector
from dotenv import load_dotenv

load_dotenv()

ctx = snowflake.connector.connect(
    user=os.getenv("SNOWFLAKE_USER"),
    password=os.getenv("SNOWFLAKE_PASSWORD"),
    account=os.getenv("SNOWFLAKE_ACCOUNT"),
)

cs = ctx.cursor()

try:
    cs.execute("SELECT current_version()")
    one_row = cs.fetchone()

    print(one_row[0])
finally:
    cs.close()

ctx.close()

8.5.1


## 웨어하우스 생성

Snowflake와 상호 작용하기 위한 SQL 명령에 익숙하다면 Python Connector 내 명령에도 익숙하실 것입니다. 명령이 어떻게 구성되어 있는지 분석해 보겠습니다.

conn은 여러분을 여러분의 Snowflake 계정과 연결하는 오브젝트임을 기억하십시오. 이를 사용하여 다음 형식을 가진 SQL 명령을 실행할 수 있습니다.

In [ ]:
conn.cursor().execute("SELECT * FROM my_table")

우선 가상 웨어하우스를 생성하겠습니다.

가상 웨어하우스에는 Snowflake를 통해 쿼리와 DML 작업을 수행하기 위해 필요한 서버가 포함되어 있습니다. 

CREATE WAREHOUSE SQL 명령을 통합하여 하나의 가상 웨어하우스를 생성할 수 있습니다.

In [ ]:
conn.cursor().execute("CREATE WAREHOUSE IF NOT EXISTS tiny_warehouse_mg")

CREATE WAREHOUSE 명령은 예상대로 웨어하우스를 생성하지만 암시적으로 해당 웨어하우스를 여러분의 세션을 위한 활성 웨어하우스로 설정하기도 합니다. 

이미 웨어하우스를 생성했다면 USE WAREHOUSE 명령을 사용하여 명시적으로 이를 활성 웨어하우스로 설정할 수 있습니다.

In [ ]:
conn.cursor().execute("USE WAREHOUSE tine_warehouse_mg")

## 데이터베이스 생성

다음 단계는 데이터베이스 생성입니다. 데이터베이스는 여러분의 데이터베이스 오브젝트가 포함되어 있는 스키마를 포함합니다.

웨어하우스를 생성했을 때와 동일한 방식으로 데이터베이스를 생성할 수 있습니다. 

다만 이번에는 CREATE DATABASE 명령을 사용하겠습니다.


In [ ]:
conn.cursor().execute("CREATE DATABASE IF NOT EXISTS my_db")
conn.cursor().execute("USE DATABASE my_db")

## 스키마 생성

스키마는 여러분의 데이터베이스 오브젝트의 그룹화입니다. 스키마에는 여러분의 테이블, 테이블에 있는 데이터 및 뷰가 포함되어 있습니다. 스키마는 데이터베이스 내에 있습니다. CREATE SCHEMA 명령으로 스키마를 생성할 수 있습니다.

CREATE SCHEMA는 또한 여러분의 세션을 위해 이를 활성 스키마로 설정하기에 명시적으로 USE SCHEMA 또한 호출할 필요가 없습니다. 이미 생성된 스키마를 사용하고 싶을 때만 사용하십시오.

In [ ]:
conn.cursor().execute("CREATE SCHEMA IF NOT EXISTS test_schema")
conn.cursor().execute("USE SCHEMA test_schema")
conn.cursor().execute("USE SCHEMA other_db.other_schema")

## 테이블 생성

웨어하우스, 데이터베이스 및 스키마를 생성했으니 데이터에 있는 데이터 조작에 필요한 모든 것이 준비되었습니다. 우선 테이블을 생성해야 합니다. CREATE TABLE 명령을 통해 생성합니다.

In [ ]:
conn.cursor().execute(
    """
    CREATE OR REPLACE TABLE test_table (
        col1 integer
        , col2 string
    )                      
"""
)

## 데이터 삽입

test_table 테이블을 생성했으니 여기에 데이터를 추가할 수 있습니다. INSERT 명령으로 추가할 수 있습니다.

In [ ]:
conn.cursor().execute(
    """
        INSERT INTO 
            test_table(col1, col2)
        VALUES
            (1, 'a')
            , (456, 'test string')
    """
)

In [ ]:
conn.cursor().execute("PUT file:///tmp/data/file* @%test_table")
conn.cursor().execute("COPY INTO test_table")

## 데이터 쿼리

당연히 언젠가 여러분의 데이터를 쿼리하고 싶을 것입니다. Python Connector 내에서 이를 쉽게 진행할 수 있습니다. print 명령으로 쉽게 테이블에서 가져온 값을 확인할 수 있습니다.

In [ ]:
col1, col2 = conn.cursor().execute("SELECT col1, col2 FROM test_table").fetchone()
print(f"{col1}, {col2}")

for col1, col2 in conn.cursor().execute("SELECT col1, col2 FROM test_table"):
    print(f"{col1}, {col2}")

In [ ]:
connection.close()